<a href="https://colab.research.google.com/github/mantoan-thi/-Aprendizagem-por-Refor-o-Profunda-para-Negocia-o-no-Mercado-de-A-es/blob/main/Aprendizagem_por_Refor%C3%A7o_Profunda_para_Negocia%C3%A7%C3%A3o_no_Mercado_de_A%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etapa 1: Instalações da bibliotecas

In [ ]:
#!pip install tensorflow-gpu==2.0.0-alpha0
!pip install tensorflow==2.0.0-beta1
!pip install tensorflow==2.0.0

In [ ]:
!pip install pandas-datareader

In [ ]:
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader

# Etapa 2: importação das bibliotecas

In [4]:
import math                              # Para construção da função sigmoid
import random                            # Para números aleatórios  
import numpy as np
import pandas as pd
import tensorflow as tf                  
import matplotlib.pyplot as plt
import pandas_datareader as data_reader   
from tqdm import tqdm_notebook,tqdm       # Para barra de progresso mais intuitivas
from collections import deque             # Para memória de re-play

In [5]:
print(tf.__version__)

2.0.0


# Etapa 3: Construção da IA para anegocição das ações

In [6]:
class AI_Trader():
  def __init__(self,state_size,action_space = 3,model_name = "AITrader"):
    self.state_size = state_size       # São as informações que estão vindo do ambiente, camada de entrada da rede neural
    self.action_space = action_space   # São as ações: comprar, vender ou não fazer nada. 
    self.memory = deque(maxlen=2000)   # Experiência de re-play
    self.model_name = model_name

    self.gamma = 0.95                  #
    self.epsilon = 1.0                 # 100% indicar que todas as ações são randomicas, sem utilizar a rede neural
    self.epsilon_final = 0.01          # 
    self.epsilon_decay = 0.995         # Decrementar os valor de espilon
    self.model = self.model_builder()

  def model_builder(self):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=32,activation = "relu",input_dim = self.state_size))
    model.add(tf.keras.layers.Dense(units=64,activation = "relu"))
    model.add(tf.keras.layers.Dense(units=128,activation = "relu"))
    model.add(tf.keras.layers.Dense(units = self.action_space,activation = 'linear'))
    model.compile(loss='mse',optimizer = tf.keras.optimizers.Adam(lr=0.001))
    return model

  def trade(self,state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  def batch_train(self,batch_size):
    batch = []
    for i in range(len(self.memory)-batch_size + 1,len(self.memory)):
      batch.append(self.memory[i])
    for state, action, reward, next_state, done in batch:
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward

      self.model.fit(state,target, epochs=1, verbose=0)
    if self.epsilon > self.epsilon_final:
      self.epsilon*=self.epsilon_decay


# Etapa 4: Pré-processamento da base de dados

Definição de funções auxiliares

Sigmoid

In [7]:
def sigmoid(x):
  return 1 /(1 + math.exp(-x))

Formatação de preços

In [8]:
def stocks_prices_format(n): # verifica se vc esta tendo ou não lucro
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

Carregador de base de dados

In [9]:
# Empresas: SABESP,    ELETROBRÁS,  ENERGISA, B3
tickers = {'AAPL','SBSP3.SA', 'ENGI3.SA', 'ELET3.SA','^BVSP'}
stock_name = list(tickers)[0]

In [10]:
def dataset_loader(stock_name):
  dataset = data_reader.DataReader(stock_name,data_source="yahoo")
  star_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset['Close']
  return close

Criador de estados

In [11]:
def state_creator(data,timestep,window_size):
  starting_id = timestep - window_size + 1
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep + 1]
  else:
    windowed_data = -starting_id * [data[0]] + list(data[0:timestep + 1])
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))
  return np.array([state])

Carregando a base de dados

In [12]:
stock_name = list(tickers)[0]
data = dataset_loader(stock_name)

# Etapa 5: Treinando a IA

Configuração dos hyper parâmentros

In [13]:
window_size = 5
episodes = 1000
batch_size = 32
data_samples = len(data)-1

Definição do modelo

In [14]:
trader = AI_Trader(window_size)

In [15]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


Loop de treinamento

In [ ]:
for episode in range(1,episodes + 1):
  print("Episode: {}/{}".format(episode,episodes))
  state = state_creator(data,0,window_size+1)
  total_profit = 0
  trader.inventory= []
  for t in tqdm(range(data_samples)):
    action = trader.trade(state)
    next_state = state_creator(data,t + 1,window_size + 1)
    reward = 0

    if action ==1:# Comprando uma ação
      trader.inventory.append(data[t])
      print("AI Trader bought: ",stocks_prices_format(data[t]))
    elif action ==2 and len(trader.inventory) > 0: # Vendendo uma ação
      buy_price = trader.inventory.pop(0)
    
      reward = max(data[t]-buy_price,0)
      total_profit+= data[t]-buy_price
      print("AI Trader soldd: ",stocks_prices_format(data[t])," Profit: ",stocks_prices_format(data[t]-buy_price))
    if t == data_samples -1:
      done = True
    else:
      done = False
    trader.memory.append((state,action,reward,next_state,done))
    state = next_state

    if done:
      print("##############################")
      print("Total profit: {}",format(total_profit))
      print("##############################")

    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
  if episode % 10 == 0:
    trader.model.save("ai_trade_{}.h5".format(episode))



Episode: 1/1000


  0%|          | 0/1239 [00:00<?, ?it/s]

AI Trader bought:  $ 30.510000
AI Trader bought:  $ 30.190001
AI Trader bought:  $ 30.799999
AI Trader soldd:  $ 30.500000  Profit:  - $ 0.010000
AI Trader bought:  $ 30.900000
AI Trader bought:  $ 30.750000
AI Trader soldd:  $ 30.650000  Profit:  $ 0.459999
AI Trader bought:  $ 30.889999
AI Trader soldd:  $ 31.049999  Profit:  $ 0.250000
AI Trader bought:  $ 30.500000
AI Trader bought:  $ 31.480000
AI Trader soldd:  $ 31.639999  Profit:  $ 0.740000
AI Trader soldd:  $ 31.990000  Profit:  $ 1.240000
AI Trader soldd:  $ 32.110001  Profit:  $ 1.220001
AI Trader soldd:  $ 31.900000  Profit:  $ 1.400000
AI Trader soldd:  $ 32.290001  Profit:  $ 0.810001


  3%|▎         | 35/1239 [00:12<10:18,  1.95it/s]

AI Trader bought:  $ 32.750000


...

AI Trader soldd:  $ 32.549999  Profit:  - $ 0.740002


 21%|██        | 258/1239 [14:54<1:01:20,  3.75s/it]

AI Trader bought:  $ 33.509998


 21%|██        | 259/1239 [14:58<1:01:01,  3.74s/it]

AI Trader soldd:  $ 34.230000  Profit:  $ 0.720001
